In [7]:
!pip install keras==2.15.0
#Import Os and Basis Libraries
import cv2
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
#Matplot Images
import matplotlib.image as mpimg
# Tensflor and Keras Layer and Model and Optimize and Loss
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import *
from tensorflow.keras.losses import BinaryCrossentropy
# import tensorflow_hub as hub
import optuna
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.applications import ResNet50, InceptionV3, Xception, VGG16
from tensorflow.keras.layers import Dense
import numpy as np
#Image Generator DataAugmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
# Warnings Remove 
import warnings 
warnings.filterwarnings("ignore")


In [8]:
# Directory containing the "Train" folder
directory = "/media/DOLPHIN_ALEXIS/Analyses_alexis/Spectrograms_datasets/dataset/_last/tests"

# List of categories (subfolder names)
categories = ["positives", "negatives"]

# Initialize lists to store filenames and categories
filenames = []
category_labels = []

# Iterate through the categories
for category in categories:
    # Path to the current category folder
    category_folder = os.path.join(directory, "train", category)
    # List all filenames in the category folder
    category_filenames = os.listdir(category_folder)
    # Append filenames and corresponding category labels
    filenames.extend(category_filenames)
    category_labels.extend([category] * len(category_filenames))

# Create DataFrame
df = pd.DataFrame({
    'filename': filenames,
    'category': category_labels
})

# Display the first few rows of the DataFrame
print(df.head())

                            filename   category
0  Exp_17_Dec_2019_1345pm-1182.8.jpg  positives
1   Exp_05_Dec_2019_1145am-182.8.jpg  positives
2  Exp_16_Jan_2020_1245pm-3208.4.jpg  positives
3  Exp_25_Nov_2019_0945am-1299.2.jpg  positives
4   Exp_01_Jan_2020_1545pm-579.6.jpg  positives


In [9]:
import tensorflow as tf

base_dir = directory

# Data_Dir
data_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")
test_dir_path = os.path.join(base_dir, "test")

# Defining data generator with Data Augmentation
data_gen_augmented = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Training Images:')
train_ds = data_gen_augmented.flow_from_directory(data_dir,
                                                  target_size = (224, 224),
                                                              batch_size = 32,
                                                              subset = 'training',
                                                              class_mode = 'categorical')


# Defining Validation_generator without Data Augmentation
data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

print('Unchanged Validation Images:')
validation_ds = data_gen.flow_from_directory(val_dir,
                                            target_size = (224, 224),
                                        batch_size = 32,
                                        subset = 'validation',
                                        class_mode = 'categorical')


# Defining Test_generator without Data Augmentation
data_test_gen = ImageDataGenerator(rescale = 1/255.)

print('Test Validation Images:')
test_ds = data_test_gen.flow_from_directory(test_dir_path,
                                            target_size = (224, 224),
                                        batch_size = 50,
                                            class_mode = 'categorical')





# THE FINETUNED MODEL NEEDS RESCALING BUT THE ORIGINAL MODEL DOESN'T NEED RESCALING


data_test_gen_nors = ImageDataGenerator(rescale = 1)
test_ds_nors = data_test_gen_nors.flow_from_directory(test_dir_path,
                                            target_size = (224, 224),
                                        batch_size = 50,
                                            class_mode = 'categorical')


Training Images:
Found 25937 images belonging to 2 classes.
Unchanged Validation Images:
Found 2129 images belonging to 2 classes.
Test Validation Images:
Found 4901 images belonging to 2 classes.
Found 4901 images belonging to 2 classes.


In [10]:
import sys
import os
current_dir = os.getcwd()

if current_dir.endswith("Models_vs_years_comparison"):
    os.chdir("../..")
    # Get the current directory
    current_dir = os.getcwd()
print(current_dir)  

# Append the relative path to the sys.path
sys.path.append(os.path.join(current_dir, "DNN_whistle_detection"))

model_folder = os.path.abspath(os.path.join(current_dir, "DNN_whistle_detection", "models"))

model_paths = os.listdir(model_folder)

models = [tf.keras.models.load_model(os.path.join(model_folder, model_path)) for model_path in model_paths]

model_names = [os.path.splitext(os.path.basename(model_path))[0] for model_path in model_paths]
print(model_names)
# print(models[-1].evaluate(test_ds))
# print(models[-1].evaluate(test_ds_nors))


/export/home1/users/zfne/emanuell/Documents/GitHub/Dolphins


2024-06-21 16:34:33.269541: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 9.00MiB (rounded to 9437184)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-06-21 16:34:33.269585: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-06-21 16:34:33.269613: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 100, Chunks in use: 100. 25.0KiB allocated for chunks. 25.0KiB in use in bin. 8.0KiB client-requested in use in bin.
2024-06-21 16:34:33.269619: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 22, Chunks in use: 22. 11.8KiB allocated for chunks. 11.8KiB in use in bin. 11.0KiB client-requested in use in bin.
2024-06-21 16:34:33.269630

ResourceExhaustedError: {{function_node __wrapped__Fill_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Fill] name: 

In [ ]:
for model_name, model in zip(model_names, models):
    test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)
    print(f"Model: {model_name}")
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

69/99 [===================>..........] - ETA: 18s - loss: 1.6032 - accuracy: 0.5084

In [ ]:
# Get the top 50 images from the test dataset
top_50_images = test_ds[0][0][:50]

# Make predictions on the top 50 images
predictions = models[3].predict(top_50_images)

# Get the predicted labels
predicted_labels = [np.argmax(prediction) for prediction in predictions]
predicted_labels_bis = predicted_labels = [(prediction[1] > prediction[0]) for prediction in predictions]
print(predicted_labels_bis)
# Get the class labels
class_labels = train_ds.class_indices
true_labels = test_ds[0][1][:50]

# Create a subplot with 5 rows and 10 columns
fig, axes = plt.subplots(5, 10, figsize=(15, 12))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Iterate over the images and their predictions
for i in range(len(top_50_images)):
    # Plot the image
    axes[i].imshow(top_50_images[i])
    axes[i].axis('off')
    
    # Set the title with the predicted label and true label
    axes[i].set_title(f"P: {int(predicted_labels[i])}, T: {np.argmax(true_labels[i])}, Pbis : {int(predicted_labels_bis[i])}")

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()
fig.savefig("test_predictions.png")

2/2 [==============================] - 0s 56ms/step
[True, True, True, False, True, True, False, True, False, True, False, True, True, False, False, False, True, False, True, True, False, True, False, True, True, False, False, False, True, False, False, False, True, True, True, False, True, False, True, True, True, True, True, False, False, True, True, False, True, False]


In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

def create_image_batch(images):
    image_batch = []
    for image in images:
        image = cv2.resize(image, (224, 224))
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        # image = image.astype(float) / 255.0
        image_batch.append(image)
    images = np.vstack(image_batch)
    print(f"Images shape :  {images.shape}")
    return images 

In [ ]:
from Predict_and_extract.utils import process_audio_file
import numpy as np

test_file_2024_new = "/media/DOLPHIN_ALEXIS/2023/Exp_01_Apr_2024_0400_channel_1.wav"
test_file_2019 = "/media/zf31/Dolphins/Sound/Exp_09_Dec_2019_0845am.wav"
test_file_2021 = "/media/zf31/Dolphins/Sound/2021/Exp_29_Apr_2021_1545pm.wav"
test_file_2023 = "/media/DOLPHIN_ALEXIS/2023/Exp_02_Feb_2023_1145_channel_1.wav"

test_files = [test_file_2024_new, test_file_2019, test_file_2021, test_file_2023]

In [ ]:
saving_folder = "./images"
batch_size = 50
start_time = 0
end_time = None
save = False
wlen = 2048
nfft = 2048
sliding_w = 0.4
cut_low_frequency = 3
cut_high_frequency = 20
target_width_px = 903
target_height_px = 677

In [ ]:
for test_file in test_files:
    print(f"Processing file {test_file}")
    images = process_audio_file(test_file, saving_folder, batch_size, start_time, end_time, save, wlen, nfft, sliding_w, cut_low_frequency, cut_high_frequency, target_width_px, target_height_px)
    images_batch = create_image_batch(images)
    for model_name, model in zip(model_names, models):
        print(f"Predicting with model {model_name}")
        predictions = model.predict(images_batch, verbose = 0)
        predicted_labels = [np.argmax(prediction) for prediction in predictions]

        fig, axes = plt.subplots(5, 10 , figsize=(15, 12))
        # Flatten the axes array for easier indexing
        axes = axes.flatten()

        for i, (image, predicted_label) in enumerate(zip(images, predicted_labels)):
            axes[i].imshow(image)
            axes[i].axis('off')
            axes[i].set_title(f"P: {int(predicted_label)}")
        plt.tight_layout()

        # Save the figure under the model and file name
        plt.savefig(f"{model_name}_{os.path.basename(test_file)}.png")
        
        plt.close()

Processing file /media/DOLPHIN_ALEXIS/2023/Exp_01_Apr_2024_0400_channel_1.wav
Images shape :  (50, 224, 224, 3)
Predicting with model model_vgg
Predicting with model CNN_Vgg_V2
Predicting with model MPFTACC95+
Predicting with model model_finetuned_vgg
Processing file /media/zf31/Dolphins/Sound/Exp_09_Dec_2019_0845am.wav
Images shape :  (50, 224, 224, 3)
Predicting with model model_vgg
Predicting with model CNN_Vgg_V2
Predicting with model MPFTACC95+
Predicting with model model_finetuned_vgg
Processing file /media/zf31/Dolphins/Sound/2021/Exp_29_Apr_2021_1545pm.wav
Images shape :  (50, 224, 224, 3)
Predicting with model model_vgg
Predicting with model CNN_Vgg_V2
Predicting with model MPFTACC95+
Predicting with model model_finetuned_vgg
Processing file /media/DOLPHIN_ALEXIS/2023/Exp_02_Feb_2023_1145_channel_1.wav
Images shape :  (50, 224, 224, 3)
Predicting with model model_vgg
Predicting with model CNN_Vgg_V2
Predicting with model MPFTACC95+
Predicting with model model_finetuned_vgg


/export/home1/users/zfne/emanuell/Documents/GitHub/Dolphins


In [ ]:

sys.path.append('/kaggle/input/')
from utility_script import *

#********************* MAIN
model_path = "/users/zfne/emanuell/Documents/GitHub/Dolphins/DNN_whistle_detection/models/model_vgg.h5"
images_path = directory + "/train/negatives"
positive_dir = "/kaggle/working/tests/train/negatives/train/negatives/positive"
negative_dir = "/kaggle/working/tests/train/negatives/train/negatives/negative"
if not os.path.exists(positive_dir):
    os.makedirs(positive_dir)
if not os.path.exists(negative_dir):
    os.makedirs(negative_dir)
csv_path = "/kaggle/working/tests/whistles.csv"

# the model
model = tf.keras.models.load_model(model_path)

# paths of all files
all_files_path = [f for f in os.listdir(images_path) if isfile(join(images_path, f))]

# lists to store data
record_names = []
positive_initial = []
positive_finish = []
class_1_scores = []

# all predictions results
predictions = []

# reading file paths 1 by 1
for file_path in all_files_path:
    
    # prediction on the given image
    prediction = predict(model, images_path, file_path)
    prediction_1= predict(model_paper, images_path, file_path)
    predictions.append([file_path, prediction, prediction_1])
    
    # if the class 1 has higher confidence than class 0
    if (prediction[0][1] > prediction[0][0]):
        
        # carry the positive image to its folder.
        copy_file(images_path, file_path, positive_dir)
        
        # storing the positive images confidences
        class_1_scores.append((prediction[0][1],prediction_1[0][1]))
        
        # preparing arrays for the csv
        record_names, positive_initial, positive_finish = prepare_csv_data(file_path,
                                                                           record_names,
                                                                           positive_initial,
                                                                           positive_finish)
    else:
        pass
        # carry the negative image to its folder.
        #copy_file(images_path, file_path, negative_dir)
        
    

#saving the csv
save_csv(record_names, positive_initial, positive_finish, class_1_scores, csv_path)